In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    # model="ibm-granite/granite-3.0-8b-instruct",
    # model="ibm-granite/granite-8b-code-instruct-128k",
    model="ibm-granite/granite-20b-code-instruct-8k",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_length":300, "temperature":0.2},
)

In [ ]:
def zeroshot_prompt(context, question, book_titles):
    """
    Creates a zero-shot prompt for the model, where the model acts as a seasoned programmer.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    - book_titles: list, list of book titles to include in the prompt
    Returns:
    - str, the formatted prompt
    """

    titles = ", ".join(book_titles)
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to generate code for an application using the context provided in the task: {context}.
    Begin by creating components based on the user question: {question}.
    Include a description of elements provided in {titles}.
    Ensure that you generate only Python code for a Jupyter Notebook using ipywidgets UI components.
    Do not output anything but the code. The output code should not exceed 300 tokens.
    """
    return prompt


In [ ]:
def get_answer_using_zeroshot(context, question, book_titles):
    """
    Generates the response from the model based on a zero-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - book_titles: list, list of book titles to include in the prompt

    Returns:
    - str, the generated result from the model
    """
    prompt = zeroshot_prompt(context, question, book_titles)
    result = model.invoke(prompt)

    return result

In [ ]:
context = "Design and develop an online bookstore UI components with minimalistic theme."
question = "Create the landing page for users visiting my bookstore. The landing page should display a header `Reader's Online Store`, a welcome message `Welcome to Reader's Verse` along with a catalog of books titles as title. Vertically align the sections."
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]

# Generate and display the UI code for the landing page
result = get_answer_using_zeroshot(context, question, book_titles)
print(f"Generated Code:\n{result}")